## Speech Recognition

1) 녹음 버튼을 눌러 녹음을 한후 'Click me' 버튼을 누르면 음성인식을 테스트.

In [1]:
from ipywebrtc import AudioRecorder, CameraStream
import torchaudio
from IPython.display import Audio
from pororo import Pororo
import IPython
import ipywidgets as widgets

asr = Pororo(task='asr', lang='ko')
tts = Pororo(task='tts', lang='multi')

camera = CameraStream(constraints={'audio': True, 'video': False})
recorder = AudioRecorder(stream=camera)

In [2]:
button = widgets.Button(description='Click me')
output = widgets.Output(layour={'border': '1px solid black'})

def on_click_callback(clicked_button: widgets.Button) -> None:
    with output:
        ret = run_asr()
        print(ret['results'][0]['speech'])
        
button.on_click(on_click_callback)
display(recorder, button, output)

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

Button(description='Click me', style=ButtonStyle())

Output()

2) Upload 버튼을 눌러 wav 파일을 업로드 해서 테스트

In [3]:
def run_asr():
    with open('recording.webm', 'wb') as f:
        f.write(recorder.audio.value)
    !ffmpeg -i recording.webm -ac 1 -f wav file.wav -y -hide_banner -loglevel panic

    return asr('file.wav')

In [4]:
upload = widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)
output2 = widgets.Output(layour={'border': '1px solid black'})

def on_upload_change(change):
    uploaded_filename = next(iter(change['owner'].value))
    content = change['owner'].value[uploaded_filename]['content']
    with open('temp.wav', 'wb') as f: f.write(content)
    with output2: display(Audio('temp.wav', autoplay=True))
    ret = asr('temp.wav')
    with output2:
        print(ret['results'][0]['speech'])
    
upload.observe(on_upload_change, names='_counter')
display(upload, output2)

FileUpload(value={}, description='Upload')

Output()